Зберемо відео з variable frame rate з "плейлістом" для демуксера вказаним нижче. Перши три фрейми мають периодичну тривалість яка в суммі доривнює п'яти секундам, останні два тривають в суммі також п'ять секунд, по дві з половиною кожний. Кінцеве відео має тривати десять секунд. Останній фрейм в "плейлісті" - заключний, він не має тривалості.

In [1]:
! cat concat.txt

file frames/black_frame.jpg
duration 1.667
file frames/white_frame.jpg
duration 1.667
file frames/black_frame.jpg
duration 1.667
file frames/black_frame.jpg
duration 2.500
file frames/white_frame.jpg
duration 2.500
file frames/white_frame.jpg


Впевнимося що сума тривалостей в полі `duration` дорівнює десяти.

In [2]:
! cat concat_count.sh

#!/bin/bash

set -eux

grep duration concat.txt | awk '{ sum += $2 } END { print sum }'


In [3]:
! bash ./concat_count.sh

+ grep duration concat.txt
+ awk '{ sum += $2 } END { print sum }'
10.001


Зберемо відео з плейліста вказаши що потрібен variable frame rate та перевіримо його тривалість з `ffprobe`.

In [4]:
! cat ./ffconcat.sh

#!/bin/bash

set -eux

ffmpeg -hide_banner -f concat -i concat.txt -fps_mode vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y
ffprobe -hide_banner -show_streams output.mp4 | grep --color "duration="


In [5]:
! bash ./ffconcat.sh

+ ffmpeg -hide_banner -f concat -i concat.txt -fps_mode vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y
Input #0, concat, from 'concat.txt':
  Duration: N/A, start: 0.000000, bitrate: N/A
  Stream #0:0: Video: mjpeg (Baseline), yuvj420p(pc, bt470bg/unknown/unknown), 1920x1080 [SAR 1:1 DAR 16:9], 25 fps, 25 tbr, 25 tbn
Stream mapping:
  Stream #0:0 -> #0:0 (mjpeg (native) -> h264 (libx264))
Press [q] to stop, [?] for help
[swscaler @ 0x140030000] deprecated pixel format used, make sure you did set range correctly
    Last message repeated 1 timesimes
[swscaler @ 0x148068000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x138008000] deprecated pixel format used, make sure you did set range correctly
[libx264 @ 0x15a006fb0] using SAR=1/1
[libx264 @ 0x15a006fb0] using cpu capabilities: ARMv8 NEON
[libx264 @ 0x15a006fb0] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x15a006fb0] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 

Тривалість яку показав `ffprobe` відрізняється від очікуваного результата(мало бути десять секунд). Розберемо відео на фрейми, написавши в оверлей таймстампу фрейма. Це можна зробити параметром `pts` у фільтрі `drawtext`.

In [6]:
! cat extract.sh

#!/bin/bash

set -eux

rm -frd ./tmp
mkdir -p ./tmp
ffmpeg -i output.mp4 -vf "drawtext=fontfile=arialbd.ttf:fontsize=24:fontcolor=white:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-tw)/2:y=h-th-10:text='%{pts\:hms}'" -vsync 0 ./tmp/output_frame_%d.png


In [7]:
! bash ./extract.sh

+ rm -frd ./tmp
+ mkdir -p ./tmp
+ ffmpeg -i output.mp4 -vf 'drawtext=fontfile=arialbd.ttf:fontsize=24:fontcolor=white:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-tw)/2:y=h-th-10:text='\''%{pts\:hms}'\''' -vsync 0 ./tmp/output_frame_%d.png
ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx2

Таймстампи мають розбіжності з "плейлістом". Так, п'ятий фрейм мав би починатись на 7.51(1.67 * 3 + 2.5) секунді, натомість він починається на дві міллісекунди раніше.

![image](./tmp/output_frame_5.png)

Висновок: роздільна здатність `ffmpeg` дорівнює 10 міллісекундам. При вказанні тривалостей в плейлісті тривалостей з меншою гранулярністю точність буде втрачена.